In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from library import *
from operator import itemgetter
import pysal
print(pysal.__version__)
random.seed(112358)

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [ ]:
zone = pd.DataFrame(columns=['zoneName', 'cityName', 'countryName', 'geometry'])

nyc_zone = pd.read_csv('nynta.csv')
nyc_zone['geometry'] = nyc_zone['the_geom'].apply(lambda x: wkt.loads(x))
nyc_zone.rename(columns={'NTAName': 'zoneName'}, inplace=True)
nyc_zone['cityName'] = 'nyc'
nyc_zone['countryName'] = 'us'
nyc_zone = nyc_zone.loc[:, ['zoneName', 'cityName', 'countryName', 'geometry']]


# !wget http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v6.geojson
la_zone = gpd.read_file('la-county-neighborhoods-v6.geojson')
la_zone.rename(columns={'name': 'zoneName'}, inplace=True)
la_zone['cityName'] = 'la'
la_zone['countryName'] = 'us'
la_zone = la_zone.loc[:, ['zoneName', 'cityName', 'countryName', 'geometry']]

mad_bcn_zone = gpd.read_file('mad_bcn_neighborhoods (1).gpkg')
mad_bcn_zone.rename(columns={'name': 'zoneName', 'city': 'cityName'}, inplace=True)
mad_bcn_zone['countryName'] = 'es'
mad_bcn_zone = mad_bcn_zone.loc[:, ['zoneName', 'cityName', 'countryName', 'geometry']]

zone = zone.append(nyc_zone)
zone = zone.append(la_zone)
zone = zone.append(mad_bcn_zone)

z = 17
data_nyc_la = pd.read_pickle('new_la_nyc_data.p')
data_mad_bcn = pd.read_pickle('new_barcelona_madrid_data.p')

country_list = list(zone['countryName'].unique()) + ['']
city_list = list(set([(_[0].lower(), _[1].lower()) for _ in list(zip(zone['cityName'], zone['countryName']))])) + ['']
feature_list = [(_, 'us') for _ in data_nyc_la.columns if not _ in ['qt_id', 'geometry', 'tag']] + [(_, 'es') for _ in data_mad_bcn.columns if not _ in ['qt_id', 'geometry', 'tag']] + ['']

zone_city_list = [(_[0], _[1].lower()) for _ in zone[['zoneName', 'cityName']].values]
with open('lite_cities.txt', 'r') as f:
    lite_cities = f.read().splitlines()
zone_city_list = [_ for _ in zone_city_list if _[0] in lite_cities]
print(len(zone_city_list))

# Feature Explorer

In [ ]:
feature_select_tab = widgets.Tab(layout=widgets.Layout(height='15%', width='25%'))
feature_run_button = widgets.Button(description="Run", disabled=True, button_style='success', layout=widgets.Layout(width='85%'))
feature_filter_button = widgets.Button(description="Filter", button_style='info', layout=widgets.Layout(width='25%'))
feature_country = widgets.Dropdown(options=country_list,
                                value='',
                                disabled=False,
                                description='Country',
                                layout=widgets.Layout(display='flex', flex_flow='row', width='65%', height='auto', justify_content='flex-start'))
feature_city = widgets.Dropdown(options=[_[0] for _ in city_list if _!=''] + [''],
                                value='',
                                disabled=True,
                                description='City',
                                layout=widgets.Layout(display='flex', flex_flow='row', width='90%', height='auto', justify_content='flex-start'))
feature_feature = widgets.Dropdown(options=[_[0] for _ in feature_list if _!='']+[''],
                                value='',
                                disabled=True,
                                description='Feature',
                                layout=widgets.Layout(display='flex', flex_flow='row', width='90%', height='auto', justify_content='flex-start'))
@feature_filter_button.on_click
def filter_country(*ignore):
    if feature_country.value != '':
        feature_run_button.disabled=False
        feature_city.disabled=False
        feature_feature.disabled=False
        feature_city.options=[_[0] for _ in city_list if (_!='') and (_[1] == feature_country.value)]
        feature_feature.options=[_[0] for _ in feature_list if (_!='') and (_[1] == feature_country.value)]
        
feature_select_tab.children = [widgets.Box([widgets.VBox([
                                                          widgets.Box([feature_country, feature_filter_button], layout=widgets.Layout(justify_content='flex-start')),
                                                          feature_city,
                                                          feature_feature, 
                                                          widgets.Box([feature_run_button], layout=widgets.Layout(justify_content='center'))])
                                           ])]
feature_select_tab.set_title(0, 'Feature Explorer')
feature_map = widgets.HTML(layout=widgets.Layout(display='flex', flex_flow='row', width='100%', height='auto'))

@feature_run_button.on_click
def feature(*ignore):
    if feature_country.value == 'us': 
        data = pd.read_pickle('new_la_nyc_data.p')
    elif feature_country.value == 'es': 
        data = pd.read_pickle('new_barcelona_madrid_data.p')
    else:
        raise 
    data_tmp = data.fillna(0)[(data.tag == feature_city.value.lower()) & (data[feature_feature.value]>0)].dropna(subset=[feature_feature.value])
    feature_map.value = Map(color_continuous_layer(Dataset.from_dataframe(data_tmp), 
                                                           feature_feature.value, 
                                                           feature_feature.value), 
                                 basemap=basemaps.positron,
                                 show_info=True)._htmlMap.html       

####################################
# -------------------------------- #
# | country |                    | #
# ----------+                    | #
# | city    |     Map            | #
# +---------+                    | #
# | feature |                    | #
# +---------+--------------------| #
# |            [Run]             | #
# -------------------------------- #
####################################
display(widgets.HBox([feature_select_tab, feature_map]))

# Twin Area Finder

In [ ]:
twin_select_tab = widgets.Tab(layout=widgets.Layout(height='250px', width='45%'))
twin_country = widgets.Dropdown(options=['us', 'es'] + [''],
                                    value='',
                                    disabled=False,
                                    description='Country',
                                    layout=widgets.Layout(display='flex', flex_flow='row', width='400px', height='auto'))
twin_country_submit = widgets.Button(description="Filter", button_style='info', layout=widgets.Layout(width='100%'))
twin_source_city = widgets.Dropdown(options=[_[0] for _ in city_list if _!='']+[''],
                                    value='',
                                    disabled=True,
                                    description='Source City',
                                    layout=widgets.Layout(display='flex', flex_flow='row', width='400px', height='auto'))
twin_target_city = widgets.Dropdown(options=[_[0] for _ in city_list if _!='']+[''],
                                    value='',
                                    disabled=True,
                                    description='Target City',
                                    layout=widgets.Layout(display='flex', flex_flow='row', width='400px', height='auto'))
twin_city_submit = widgets.Button(description="Filter", button_style='success', layout=widgets.Layout(width='100%', height='auto'))
twin_city_line = widgets.HTML(value='<hr>', layout=widgets.Layout(width='520px', justify_content='center'))
twin_source_zone = widgets.Dropdown(disabled=True,
                                    description='Source Zone',
                                    layout=widgets.Layout(display='flex', flex_flow='row', width='400px', height='auto'))
twin_select_tab.children = [widgets.Box([widgets.VBox([widgets.HBox([twin_country, widgets.Box([twin_country_submit])]),
                                                       widgets.HBox([
                                                           widgets.VBox([twin_source_city, twin_target_city]),
                                                           widgets.Box([twin_city_submit])
                                                       ]),
                                                       twin_city_line,
                                                       twin_source_zone])
                                           ])]
twin_select_tab.set_title(0, 'Twin Area')
@twin_country_submit.on_click
def filter_country(*ignore):
    if twin_country.value != '':
        twin_source_city.disabled=False
        twin_target_city.disabled=False
        if twin_country.value == 'us':
            twin_source_city.options=['nyc', 'la']
            twin_target_city.options=['la', 'nyc']
        if twin_country.value == 'es':
            twin_source_city.options=['madrid', 'barcelona']
            twin_target_city.options=['barcelona', 'madrid']

twin_config_tab = widgets.Tab(layout=widgets.Layout(height='250px', width='45%'))
twin_k = widgets.IntSlider(value=2000, 
                           min=2, 
                           max=10000, 
                           description='Top K Areas', 
                           layout=widgets.Layout(display='flex', flex_flow='row', lign_items='flex-end',
                                                 width='400px'))
twin_config_line = widgets.HTML(value='<br/>', layout=widgets.Layout(width='400px', justify_content='center'))
twin_score_range = widgets.FloatRangeSlider(
    value=[0, 1],
    min=-1,
    max=1,
    step=0.05,
    description='Similarity Score',
    style={'description_width': '100px'},
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=widgets.Layout(display='inlineflex', flex_flow='row', lign_items='flex-end', width='400px', height='auto'))
twin_config_tab.children = [widgets.Box([widgets.VBox([twin_k, twin_config_line, twin_score_range])])]
twin_config_tab.set_title(0, 'Search Configuration')

twin_feature_tab = widgets.Tab(layout=widgets.Layout(width='45%', height='250px'))
twin_feature_tab_name = ['Demographics', 'PoI', 'Financial', 'HumanMobility', 'RoadTraffic']
twin_feature_tab.children = [widgets.SelectMultiple(
    rows=8,
    layout=widgets.Layout(display='none',
                          flex_flow='row',
                          align_items='stretch',
                          width='auto',
                          height='auto'),
                          disabled=False) 
    for name in twin_feature_tab_name]
    
for i in range(len(twin_feature_tab_name)):
    twin_feature_tab.set_title(i, twin_feature_tab_name[i])
    
twin_selected_feature_tab = widgets.Tab(layout=widgets.Layout(width='45%', height='250px'))
twin_selected_feature_list = widgets.SelectMultiple(rows=10, 
                                                    layout=widgets.Layout(width='auto', height='auto', disabled=True, display='none'))
twin_selected_feature_tab.children = [twin_selected_feature_list]
twin_selected_feature_tab.set_title(0, 'Selected Features')

def twin_feature_select_update(*ignore):
    selected = []
    for child in twin_feature_tab.children:
        selected.append(list(child.value))
    twin_selected_feature_list.options = sum(selected, [])
for _ in twin_feature_tab.children:
    _.observe(twin_feature_select_update, 'value')


twin_run_button = widgets.Button(description="Run", button_style='warning', layout=widgets.Layout(width='90%'))

map_source_hist_tab = widgets.Tab(layout=widgets.Layout(width='90%', height='100%', display='none'))
map_source = widgets.HTML(layout=widgets.Layout(width='30%', height='90%'))

hist_features = [widgets.Dropdown(description='Feature',
                                  disabled=False,
                                  layout=widgets.Layout(justify_content='space-around', width='85%', height='35.5%')) for _ in range(6)]
hist = [Output(layout=widgets.Layout(width='100%', height='100%', justify_content='space-around')) for _ in range(6)]
map_source_hist_tab.children = [widgets.HBox([widgets.HBox([map_source,
                                                            widgets.VBox([widgets.HBox([widgets.VBox([hist_features[0], hist[0]], layout=widgets.Layout(justify_content='space-around')), 
                                                                                        widgets.VBox([hist_features[1], hist[1]], layout=widgets.Layout(justify_content='space-around')),
                                                                                        widgets.VBox([hist_features[2], hist[2]], layout=widgets.Layout(justify_content='space-around'))]), 
                                                                          widgets.HBox([widgets.VBox([hist_features[3], hist[3]], layout=widgets.Layout(justify_content='space-around')),
                                                                                        widgets.VBox([hist_features[4], hist[4]], layout=widgets.Layout(justify_content='space-around')),
                                                                                        widgets.VBox([hist_features[5], hist[5]], layout=widgets.Layout(justify_content='space-around'))])],
                                                                         layout=widgets.Layout(justify_content='space-around')
                                                                        )])])]
map_source_hist_tab.set_title(0, 'Source')

map_twin_tab = widgets.Tab(layout=widgets.Layout(width='90%', height='100%', display='none'))
map_twin = widgets.HTML(layout=widgets.Layout(width='100%', height='100%'))
map_twin_tab.children = [map_twin]
map_twin_tab.set_title(0, 'Top K Twin Areas')

map_hotspot_tab = widgets.Tab(layout=widgets.Layout(width='90%', height='100%', display='none'))
map_hotspot = widgets.HTML(layout=widgets.Layout(width='100%', height='100%'))
map_hotspot_tab.children = [map_hotspot]
map_hotspot_tab.set_title(0, 'Hot Spot Areas')

@twin_city_submit.on_click
def filter_source_zone(*ignore):
    twin_source_zone.disabled=False
    twin_source_zone.options = [zone_city_list[_][0] for _ in range(len(zone_city_list)) if (_!='') and (zone_city_list[_][1] == twin_source_city.value.lower())]+['']
    twin_source_zone.value = ''

@twin_city_submit.on_click
def feature_diff_country(*ignore):
    for i, _ in enumerate(twin_feature_tab.children):
        name = twin_feature_tab_name[i]
        if twin_country.value == 'es':
            _.options = [_[0] for _ in feature_list if (_!='') and _[1] == 'es' and (_[0].startswith(name))]
        elif twin_country.value == 'us':
            _.options = [_[0] for _ in feature_list if (_!='') and _[1] == 'us' and (_[0].startswith(name))]
        else:
            pass
        _.layout = widgets.Layout(display='flex', flex_flow='row', align_items='stretch',width='auto',height='auto')
    twin_selected_feature_list.layout = widgets.Layout(width='auto', height='auto', disabled=True)

@twin_run_button.on_click
def hist_feature_diff_country(*ignore):
    for i, _ in enumerate(hist_features):
        _.options = twin_selected_feature_list.options
        if len(twin_selected_feature_list.options) == 0:
            pass
        elif len(twin_selected_feature_list.options) < 6:
            if i < len(twin_selected_feature_list.options):
                _.value = twin_selected_feature_list.options[i]
            else:
                _.value = twin_selected_feature_list.options[-1]
        else:
            _.value = twin_selected_feature_list.options[i]
        

@twin_run_button.on_click
def turn_on_map_tab(*ignore):
    map_source_hist_tab.layout = widgets.Layout(width='90%', height='100%')
    map_twin_tab.layout = widgets.Layout(width='90%', height='100%')
    map_hotspot_tab.layout = widgets.Layout(width='90%', height='100%')

    
    
@twin_run_button.on_click
def twin(*ignore):
    if twin_country.value == 'es':
        data = pd.read_pickle('new_barcelona_madrid_data.p')
    else:
        data = pd.read_pickle('new_la_nyc_data.p')
    
    zone_copy = zone.copy()
    zone_copy = zone_copy[zone_copy.zoneName == twin_source_zone.value]
    source_qtid = qt_convert(zone_copy.geometry.values[0].centroid.y, 
                             zone_copy.geometry.values[0].centroid.x,
                             zoom=z)
    source_location = pd.DataFrame([source_qtid], columns=['qt_id'])
    source_location['geometry'] = source_location['qt_id'].apply(lambda x: quad_to_poly(x))
    
    pc_fname = str(hash(tuple(list(sorted(list(twin_selected_feature_list.options)))+[twin_source_city.value.lower(), twin_target_city.value.lower()]))) 
    if not os.path.isdir('Data/'):
        os.mkdir('Data/')
    if os.path.isfile(f'Data/{pc_fname}.p'):
        reconstructed_data = pickle.load(open(f"Data/{pc_fname}.p", "rb" ))
    else:
        reconstructed_data = calc_ens_pc(data = data, 
                                         selected_col_prep = list(twin_selected_feature_list.options), 
                                         source = twin_source_city.value.lower(), 
                                         target = twin_target_city.value.lower(), 
                                         n_ens_member = 5, 
                                         seed = 112358)
        pickle.dump(reconstructed_data, open(f"Data/{pc_fname}.p", "wb"))

    twin_result = similarity_score(reconstructed_data, 
                                   source_qtid, 
                                   source=twin_source_city.value.lower(), 
                                   target=twin_target_city.value.lower()) 
    twin_result['geometry'] = twin_result['qt_id'].apply(lambda x: quad_to_poly(x))
    twin_result_filter = twin_result[(twin_result['similarity_skill_score']>=twin_score_range.value[0]) & (twin_result['similarity_skill_score']<=twin_score_range.value[1])]
    
    map_source.value = Map(layers=[Layer(source=Dataset.from_dataframe(source_location), 
                                         style='''
                                               color: red
                                               strokeColor: red
                                               strokeWidth: 10
                                               ''',
                                         popup={'hover': [{'title': 'Quadkey', 'value': '$qt_id'}],
                                                'click': {'title': 'Quadkey', 'value': '$qt_id'}},
                                         legend={'type': 'color-category',
                                                 'prop': 'color',
                                                 'title': f'[Source] {twin_source_city.value.upper()}',
                                                 'description': 'Source Grid Cell'}, 
                                         context=cc),
                                   Layer(source=Dataset.from_dataframe(zone[zone.zoneName == twin_source_zone.value]), 
                                         style={'color': 'opacity(red, 0.1)'}, 
                                         popup={'hover': ['$zoneName', '$cityName', '$countryName']},
                                         context=cc)],
                           show_info=True, 
                           viewport={'zoom': 9.5})._htmlMap.html
    
    map_twin.value = Map(layers=[Layer(source=Dataset.from_dataframe(twin_result_filter.sort_values(by='similarity_skill_score', ascending=False).head(twin_k.value)), 
                                         style='''
                                               @val: $similarity_skill_score
                                               @col: OrYel
                                               @val_min: globalPercentile(@val, 1)
                                               @val_max: globalPercentile(@val, 99)
                                               @alpha: mul(0.75, div(@val, globalMax(@val))) + 0.1
                                               @stroke_alpha: @alpha+0.1
                                               color: opacity(ramp(linear(@val, @val_min, @val_max), @col), @alpha)
                                               strokeWidth: 6
                                               strokeColor: opacity(ramp(linear(@val, @val_min, @val_max), @col), @stroke_alpha)
                                               ''', 
                                         popup={'hover': [{'title': 'Quadkey', 'value': '$qt_id'}],
                                                'click': [{'title': 'Quadkey', 'value': '$qt_id'},
                                                          {'title': 'Similarity Skill Score', 'value': '$similarity_skill_score'}]},
                                         legend={'type': 'color-continuous-polygon',
                                                 'prop': 'color',
                                                 'title': f'[Twin Area] {twin_target_city.value.upper()}',
                                                 'description': f'Top {twin_k.value} Twin Grid Cell'}, 
                                         context=cc)], 
                           basemap=basemaps.positron,
                           context=cc)._htmlMap.html
    
    twin_result_copy = twin_result_filter.copy()
    twin_result_copy = twin_result_copy[twin_result_copy['similarity_skill_score']>=0]
    Contiguity = pysal.weights.KNN.from_dataframe(twin_result_copy, geom_col='geometry', ids='qt_id', k=8)
    Contiguity.transform = 'r'
    lag = pysal.lag_spatial(Contiguity, twin_result_copy['similarity_skill_score'])
    LISA = pysal.Moran_Local(lag, Contiguity, permutations=999)
    hh_ll_pval = LISA.p_sim <= 0.01
    hh = (lag >= lag.mean()) & (twin_result_copy['similarity_skill_score'] >= twin_result_copy['similarity_skill_score'].mean()) & hh_ll_pval
    ll = (lag < lag.mean()) & (twin_result_copy['similarity_skill_score'] < twin_result_copy['similarity_skill_score'].mean()) & hh_ll_pval
    hotspot = twin_result_copy.copy()
    hotspot['SpatialLag'] = lag
    hotspot['pval'] = LISA.p_sim
    hotspot['isHotSpot'] = hh
    hotspot['isColdSpot'] = ll
    
    map_hotspot.value = Map(layers=[Layer(source=Dataset.from_dataframe(hotspot[hotspot['isHotSpot'] == True]), 
                                         style='''
                                               color: opacity(red, 0.5)
                                               strokeColor: opacity(red, 0.5)
                                               strokeWidth: 0
                                               ''',
                                         popup={'hover': [{'title': 'Quadkey', 'value': '$qt_id'}],
                                                'click': [{'title': 'similarity_skill_score', 'value': '$similarity_skill_score'},
                                                          {'title': 'SpatialLag', 'value': '$SpatialLag'},
                                                          {'title': 'pval', 'value': '$pval'},
                                                          {'title': 'isHotSpot', 'value': '$isHotSpot'},
                                                          {'title': 'isColdSpot', 'value': '$isColdSpot'}]},
                                         legend={'type': 'color-category',
                                                 'prop': 'color',
                                                 'title': f'[Hot Spot] {twin_target_city.value.upper()}',
                                                 'description': 'Hot Spots Area'}, 
                                         context=cc)],
                           show_info=True, 
                           viewport={'zoom': 9.5})._htmlMap.html


def histogram(*ignore):
    if twin_country.value == 'es':
        data = pd.read_pickle('new_barcelona_madrid_data.p')

    else:
        data = pd.read_pickle('new_la_nyc_data.p')  
        
    zone_copy = zone.copy()
    zone_copy = zone_copy[zone_copy.zoneName == twin_source_zone.value]
    source_qtid = qt_convert(zone_copy.geometry.values[0].centroid.y, 
                             zone_copy.geometry.values[0].centroid.x,
                             zoom=17)
    
    f_size=(4, 3.5)
    with hist[0]:
        clear_output(wait=True)
        hist_data = data.copy()
        f, ax = plt.subplots(1,1,figsize=f_size)
        ax.clear()
        try:
            x = hist_data[hist_data['qt_id']==source_qtid][hist_features[0].value].values[0]
            if not np.isnan(x):
                sns.distplot(hist_data[hist_features[0].value].dropna(), ax=ax, axlabel=False)
                ax.axvline(x, color='red')
            else:
                ax.clear()
        except:
            ax.clear()
        f.tight_layout()
        plt.subplots_adjust(top=0.75, wspace = 0, hspace = 0.4, right = 1)
        plt.show()
        
    with hist[1]:
        clear_output(wait=True)
        hist_data = data.copy()
        f, ax = plt.subplots(1,1,figsize=f_size)
        ax.clear()
        try:
            x = hist_data[hist_data['qt_id']==source_qtid][hist_features[1].value].values[0]
            if not np.isnan(x):
                sns.distplot(hist_data[hist_features[1].value].dropna(), ax=ax, axlabel=False)
                ax.axvline(x, color='red')
            else:
                ax.clear()
        except:
            ax.clear()
        f.tight_layout()
        plt.subplots_adjust(top=0.75, wspace = 0, hspace = 0.4, right = 1)
        plt.show()
        
    with hist[2]:
        clear_output(wait=True)
        hist_data = data.copy()
        f, ax = plt.subplots(1,1,figsize=f_size)
        ax.clear()
        try:
            x = hist_data[hist_data['qt_id']==source_qtid][hist_features[2].value].values[0]
            if not np.isnan(x):
                sns.distplot(hist_data[hist_features[2].value].dropna(), ax=ax, axlabel=False)
                ax.axvline(x, color='red')
            else:
                ax.clear()
        except:
            ax.clear()
        f.tight_layout()
        plt.subplots_adjust(top=0.75, wspace = 0, hspace = 0.4, right = 1)
        plt.show()

    with hist[3]:
        clear_output(wait=True)
        hist_data = data.copy()
        f, ax = plt.subplots(1,1,figsize=f_size)
        ax.clear()
        try:
            x = hist_data[hist_data['qt_id']==source_qtid][hist_features[3].value].values[0]
            if not np.isnan(x):
                sns.distplot(hist_data[hist_features[3].value].dropna(), ax=ax, axlabel=False)
                ax.axvline(x, color='red')
            else:
                ax.clear()
        except:
            ax.clear()
        f.tight_layout()
        plt.subplots_adjust(top=0.75, wspace = 0, hspace = 0.4, right = 1)
        plt.show()
        
    with hist[4]:
        clear_output(wait=True)
        hist_data = data.copy()
        f, ax = plt.subplots(1,1,figsize=f_size)
        ax.clear() 
        try:
            x = hist_data[hist_data['qt_id']==source_qtid][hist_features[4].value].values[0]
            if not np.isnan(x):
                sns.distplot(hist_data[hist_features[4].value].dropna(), ax=ax, axlabel=False)
                ax.axvline(x, color='red')
            else:
                ax.clear()
        except:
            ax.clear()
        f.tight_layout()
        plt.subplots_adjust(top=0.75, wspace = 0, hspace = 0.4, right = 1)
        plt.show()
        
    with hist[5]:
        clear_output(wait=True)
        hist_data = data.copy()
        f, ax = plt.subplots(1,1,figsize=f_size)
        ax.clear()
        try:
            x = hist_data[hist_data['qt_id']==source_qtid][hist_features[5].value].values[0]
            if not np.isnan(x):
                sns.distplot(hist_data[hist_features[5].value].dropna(), ax=ax, axlabel=False)
                ax.axvline(x, color='red')
            else:
                ax.clear()  
        except:
            ax.clear()     
        f.tight_layout()
        plt.subplots_adjust(top=0.75, wspace = 0, hspace = 0.4, right = 1)
        plt.show()
            
        
#############
## Observe ##
#############    
# \\\\dj\\\ #
# |       | #
# |<O>-<O>| #
# |   <   | #
# \   V  /  #
#  \____/   #
#############
for _ in hist_features:
    _.observe(histogram, 'value')
twin_run_button.on_click(histogram)
###########################################################################################################################################################################################



display(widgets.VBox([widgets.HBox([twin_select_tab, 
                            twin_config_tab]),
              widgets.HBox([twin_feature_tab, 
                            twin_selected_feature_tab]),
              twin_run_button,
              map_source_hist_tab,
              map_twin_tab,
              map_hotspot_tab]))
